In [ ]:
conda install -c conda-forge pyscipopt


In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import matplotlib
import mosek
matplotlib.use('TkAgg')
from matplotlib.patches import Rectangle


                                                                                                    
#cp.settings.ERROR = [cp.settings.USER_LIMIT]                                                                            
#cp.settings.SOLUTION_PRESENT = [cp.settings.OPTIMAL, cp.settings.OPTIMAL_INACCURATE, cp.settings.SOLVER_ERROR]  

def MIMPC_2D( x0=np.array([0, .5,0,0]),ol=np.array([[1,1,7,9,10],[3,-1,2,5,5]]),obs_size=np.array([[1,1,1,1,2],[2,2,2,2,1]])): 
    
    #Fixed parameters
    N_obs_max=30
    M=20



    # Model data
    A = np.matrix([[1, 0, 0.1, 0],[0, 1, 0, 0.1],[0, 0, 1, 0],[0, 0, 0, 1]])
    B = 0.1*np.matrix([[0, 0],[0, 0],[1, 0],[0, 1]])

    nx = A.shape[1]#% Number of states
    nu = B.shape[1]#% Number of inputs

    # MPC data
    Q = 100*np.identity(nx)
    R = 0*np.identity(nu)
    N = 3

    lb_x=np.array([-15,-15,-8,-8])
    ub_x=-lb_x

    lb_u=np.array([-5,-5])
    ub_u=-lb_u

    # If no arguments are passed to the function, default intialisation below
   

    n_obs=ol.shape[1]
    N_obs_max=n_obs

    if n_obs<N_obs_max:
        ol=np.concatenate((ol, M*np.ones((2,N_obs_max-n_obs))),axis=1)
        obs_size=np.concatenate((obs_size, np.zeros((2,N_obs_max-n_obs))),axis=1)
    
    ou=ol+obs_size



    # Decision variables
    x = cp.Variable((nx,N+1))
    u = cp.Variable((nu,N))

    #s_x_l=cp.Variable((nx,N))
    #s_x_u=cp.Variable((nx,N))
    #s_u_l=cp.Variable((nu,N))
    #s_u_u=cp.Variable((nu,N))

    #Slack and binary varibles definition
    #s_oa_l=[cp.Variable((2,N)) for _ in range(N_obs_max)]
    #s_oa_u= [cp.Variable((2,N)) for _ in range(N_obs_max)]
    #s_oa_side=[cp.Variable((2,N)) for _ in range(N_obs_max)]
    b_l=[cp.Variable((2,N),boolean=True) for _ in range(N_obs_max)]
    b_u=[cp.Variable((2,N),boolean=True) for _ in range(N_obs_max)]
    
    # MPC problem setup
    Model=[]
    constraints = [x[:,1]==x0]#, s_x_l>=0, s_x_u>=0, s_u_l>=0, s_u_u>=0]
                
    objective   = 0
    
    for k in range(N):
        objective = objective + cp.norm(Q@x[:,k],'inf') + cp.norm(R@u[:,k],'inf')
        constraints += [x[:,k+1]== A@x[:,k] + B@u[:,k]]
        
        state_constr=[]
        input_constr=[]
        
        constraints+=[lb_x<=x[:,k+1],#-s_x_l[:,k],
                    ub_x>=x[:,k+1]]#+s_x_u[:,k]]
    
        constraints+=[lb_u<=u[:,k],#-s_u_l[:,k],
                    ub_u>=u[:,k]]#+s_u_u[:,k]]

        obs_avoid=[]            
        for i in range (N_obs_max):
            constraints+=[
                        x[0:2,k+1]<=ol[:,i]+M*b_l[i][:,k],
                        #x[0:2,k+1]+s_oa_l[i][:,k]==ol[:,i]+M*b_l[i][:,k],
                        #x[0:2,k+1]-s_oa_u[i][:,k]==ou[:,i]-M*b_u[i][:,k],
                        x[0:2,k+1]>=ou[:,i]-M*b_u[i][:,k],    
                        b_l[i][0,k]+b_l[i][1,k]+b_u[i][0,k]+b_u[i][1,k]<=2*2-1]               
                    #   b_l[i][0,k]+b_l[i][1,k]+b_u[i][0,k]+b_u[i][1,k]+s_oa_side[i][0,k]==2*2-1,
                    #   s_oa_l[i]>=0, s_oa_u[i]>=0, s_oa_side[i]>=0]

        #constraints+= [Model, obs_avoid, state_constr, input_constr] 
    
    
    objective = objective + 100*cp.norm(Q@x[:,k+1],1)
    problem = cp.Problem(cp.Minimize(objective), constraints)
    
    
    problem.solve(solver=cp.MOSEK, save_file='mosek_model.cip',verbose=True)#, scip_params={'branching/scorefunc':'p'})

    print("Status",problem.status)
    print("The optimal value is ", problem.value)
    print("A solution x is ")
    print("Solver stats", problem.solver_stats)
    x_sol=x.value
    u_sol=u.value
    b_l_sol=[b_l[i].value for i in range(N_obs_max)]
    b_u_sol=[b_u[i].value for i in range(N_obs_max)]

    return x_sol, u_sol, b_l_sol, b_u_sol, ol, obs_size,problem
    #print(x.value)
    
    
 ## Generating problem and solution   
[x_sol, u_sol, b_l_sol, b_u_sol, ol, obs_size,problem]=MIMPC_2D()
print(x_sol)
fig=plt.figure()
n_obs=obs_size.shape[-1]
plt.plot(x_sol[0,:], x_sol[1,:],'--')
for i in range(n_obs):
    plt.gca().add_patch(Rectangle((ol[0,i],ol[1,i]),obs_size[0,i],obs_size[1,i]))
plt.show()


                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 20 12:54:11 PM: Your problem has 82 variables, 61 constraints, and 0 parameters.
(CVXPY) Jun 20 12:54:11 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 20 12:54:11 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 20 12:54:11 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 20 12:54:11 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jun 20 12:54:11 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

In [ ]:
from pyscipopt import Model, quicksum
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.patches import Rectangle

def MIMPC_2D( x0=np.array([10, 6.5,0,0]),ol=np.array([[1,1,3,9,10],[3,-1,2,5,5]]),obs_size=np.array([[1,1,1,1,2],[2,2,2,2,1]])): 

    m=Model()
    # Model data
    A = np.matrix([[1, 0, 0.1, 0],[0, 1, 0, 0.1],[0, 0, 1, 0],[0, 0, 0, 1]])
    B = 0.1*np.matrix([[0, 0],[0, 0],[1, 0],[0, 1]])
    N=30
    M=20
    # N_obs_max=10
    nx = A.shape[1]#% Number of states
    nu = B.shape[1]#% Number of inputs

    # Decision variables
    x,u,tx,tu,sxl,sxu,sul,suu={},{},{},{},{},{},{},{}
    Q = 100*np.identity(nx)
    R = 0*np.identity(nu)

    lb_x=np.array([-15,-15,-8,-8])
    ub_x=-lb_x

    lb_u=np.array([-5,-5])
    ub_u=-lb_u
    n_obs=ol.shape[1]
    N_obs_max=n_obs
    if n_obs<N_obs_max:
        ol=np.concatenate((ol, M*np.ones((2,N_obs_max-n_obs))),axis=1)
        obs_size=np.concatenate((obs_size, np.zeros((2,N_obs_max-n_obs))),axis=1)
    
    ou=ol+obs_size

    for j in range(N+1):
        for i in range(nx):
            x[i,j] = m.addVar( vtype='C',name="x(%s,%s)"%(i,j),lb=None,ub=None)
            
            #if j<N:
                #sxl[i,j] = m.addVar(vtype='C',name="sxl(%s,%s)"%(i,j))
                #sxu[i,j] = m.addVar(vtype='C',name="sxu(%s,%s)"%(i,j))

    for j in range(N):
        tx[j] = m.addVar( vtype='C',name="tx(%s)"%(j),lb=None,ub=None)
        tu[j] = m.addVar( vtype='C',name="tu(%s)"%(j),lb=None,ub=None)
        for i in range(nu):
            u[i,j] = m.addVar( vtype='C',name="u(%s,%s)"%(i,j),lb=None,ub=None)
            #sul[i,j]=m.addVar( vtype='C',name="sul(%s,%s)"%(i,j))
            #suu[i,j]=m.addVar( vtype='C',name="suu(%s,%s)"%(i,j))

    #Slack and binary varibles definition
    s_oal,s_oau,s_oas,bl,bu={},{},{},{},{}
    for i in range(2):
        for j in range(N):
            for k in range(N_obs_max):
                #s_oal[i,j,k] = m.addVar(vtype='C',name="s_oal(%s,%s,%s)"%(i,j,k))
                #s_oau[i,j,k] = m.addVar(vtype='C',name="s_oau(%s,%s,%s)"%(i,j,k))
                #s_oas[i,j,k] = m.addVar(vtype='C',name="s_oas(%s,%s,%s)"%(i,j,k))
                bl[i,j,k] = m.addVar(vtype='B',name="bl(%s,%s,%s)"%(i,j,k))
                bu[i,j,k] = m.addVar(vtype='B',name="bu(%s,%s,%s)"%(i,j,k))


    # Dynamics constraints and state-input constraints
    cost=0
    for k in range(N+1):
        if k<N:
            cost+=tx[k]+tu[k]
        for i in range(nx):
            if k==0:
                m.addCons(x[i,k]==x0[i])
            else:
                
                m.addCons((1+100*(k==N))*Q[i,i]*x[i,k]<=tx[k-1])
                m.addCons(-(1+100*(k==N))*Q[i,i]*x[i,k]<=tx[k-1])
                m.addCons(x[i,k]==quicksum(A[i,j]*x[j,k-1] for j in range(nx))+quicksum(B[i,j]*u[j,k-1] for j in range(nu)))
                #m.addCons(sxl[i,k-1]>=0)
                #m.addCons(sxu[i,k-1]>=0)
                m.addCons(x[i,k]<=ub_x[i])
                m.addCons(x[i,k]>=lb_x[i])
                if i<nu:#assuming nu<nx
                    #m.addCons(sul[i,k-1]>=0)
                    #m.addCons(suu[i,k-1]>=0)
                    m.addCons(R[i,i]*u[i,k-1]<=tu[k-1])
                    m.addCons(-R[i,i]*u[i,k-1]<=tu[k-1])
                    m.addCons(u[i,k-1]<=ub_u[i])
                    m.addCons(u[i,k-1]>=lb_u[i])
                for l in range(N_obs_max):
                    if i<2:
                        m.addCons(x[i,k]<=ol[i,l]+M*bl[i,k-1,l])
                        m.addCons(x[i,k]>=ou[i,l]-M*bu[i,k-1,l])
                        m.addCons(bl[0,k-1,l]+bl[1,k-1,l]+bu[0,k-1,l]+bu[1,k-1,l]<=2*2-1)

    m.setObjective(cost, "minimize")
    #m.setParam("lp/disablecutoff","1")
   # m.setHeuristics(3)
   # m.setPresolve(3)
    #nodelist=[]
    
    #m.optimize()
    
    m.writeProblem("model.cip")
    Variables=m.getVars()
    # x_sol=[x0]
    # u_sol=[]
    # for i in range(N):
    #     x_sol.append([])
    #     u_sol.append([])
    #     for j in range(nx):
    #         x_sol[i+1].append(m.getVal(x[j,i+1]))
    #     for j in range (nu):
    #         u_sol[i].append(m.getVal(u[j,i]))

    # a=2


    # x_sol=np.array(x_sol)
    # u_sol=np.array(u_sol)
    # return x_sol,u_sol, ol, obs_size,m
    return m,bl,bu
            
        
        
# [x_sol,u_sol,ol, obs_size,m]=MIMPC_2D()
[m,bl,bu]=MIMPC_2D()

# fig=plt.figure()
# n_obs=obs_size.shape[-1]
# plt.plot(x_sol[:,0], x_sol[:,1],'--')
# for i in range(n_obs):
#     plt.gca().add_patch(Rectangle((ol[0,i],ol[1,i]),obs_size[0,i],obs_size[1,i]))
# plt.show()

#e()
#x_sol=m.getVal(m.data[0])
# u_sol=m.getVal(u)
a=1











In [2]:
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE
from sklearn import manifold
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import networkx as nx
import os
import math
from time import time

#EventHandler for collecting node information

class LPstatEventhdlr(Eventhdlr):
    """PySCIPOpt Event handler to collect data on LP events."""

    vars = {}

    def collectNodeInfo(self, firstlp=True):
        objval = self.model.getSolObjVal(None)
        #if abs(objval) >= self.model.infinity():
        #   return

        LPsol = {}
        if self.vars == {}:
            self.vars = self.model.getVars(transformed=False)
        for var in self.vars:
            solval = self.model.getSolVal(None, var)
            # store only solution values above 1e-6
            # # if abs(solval) > 1e-6:
            if(var.vtype())=='BINARY':
                LPsol[var.name] = self.model.getSolVal(None, var)

        # skip duplicate nodes
        #if self.nodelist and LPsol == self.nodelist[-1].get("LPsol"):
            #return
        node = self.model.getCurrentNode()
        vars=self.model.getVars()
        lb={}
        ub={}

        for v in vars:
            if v.vtype()=='BINARY':
                lb[v.name]=[v,v.getLbLocal()]
                ub[v.name]=[v,v.getUbLocal()]

    
        openNodes=self.model.getOpenNodes()
        openNodesNumber=[]
        Open_nodes_lb={}
        for n in range(len(openNodes[0])):
            openNodesNumber.append(openNodes[0][n].getNumber())
            # for v in vars:
            #     if v.vtype()=='BINARY':
            #         Open_nodes_lb[v.name]=[v,v.getLbLocal()]
            #         Open_nodes_ub[v.name]=[v,v.getUbLocal()]
                    
        
        if node.getNumber() != 1:
            parentnode = node.getParent()
            parent = parentnode.getNumber()
            
        else:
            parent = 1
        depth = node.getDepth()
        children=self.model.getNChildren()
        
 
        age = self.model.getNNodes()
        condition = math.log10(self.model.getCondition())
        iters = self.model.lpiGetIterations()
        pb = self.model.getPrimalbound()
        if pb >= self.model.infinity():
            pb = None

        nodedict = {
            "number": node.getNumber(),
            "type":node.getType(),
            "openNodes":openNodes,
            "openNodesNumber":openNodesNumber,
            "LPsol": LPsol,
            "objval": objval,
            "parent": parent,
            "node":node,
            "age": age,
            "depth": depth,
            "first": firstlp,
            "condition": condition,
            "iterations": iters,
            "lb":lb,
            "ub":ub,
            # "variables": self.model.getNVars(),
            "constraints": self.model.getConss(),
            "rows": self.model.getNLPRows(),
            "primalbound": pb,
            "dualbound": self.model.getDualbound(),
            "time": self.model.getSolvingTime()
        }
        # skip 0-iterations LPs (duplicates?)
        if firstlp:
            self.nodelist.append(nodedict)
        #elif iters > 0:
        #   prevevent = self.nodelist[-1]
        #    if nodedict["number"] == prevevent["number"] and not prevevent["first"]:
                # overwrite data from previous LP event
        #        self.nodelist[ -1] = nodedict
        #    else:
        #        self.nodelist.append(nodedict)

    def eventexec(self, event):

        self.collectNodeInfo(firstlp=True)
        
        # if event.getType() == SCIP_EVENTTYPE.FIRSTLPSOLVED:
        #    self.collectNodeInfo(firstlp=True)
        # elif event.getType() == SCIP_EVENTTYPE.LPSOLVED:
        #    self.collectNodeInfo(firstlp=True)
        # else:
        #    print("unexpected event:" + str(event))
        return {}
    #def eventexit(self):
    #    self.model.dropEvent(SCIP_EVENTTYPE.NODEFEASIBLE, self)


    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.NODEFEASIBLE, self)



    

In [ ]:
# model=Model()
model=m
nodelist=[]
eventhdlr = LPstatEventhdlr()
eventhdlr.nodelist = nodelist
# model.readProblem('/home/luirusso/LearnWSL/model.cip')
model.disablePropagation()
model.setSeparating(3)
model.setPresolve(3)
#model.setHeuristics(3)
#model.setParam("lp/disablecutoff","1")
model.includeEventhdlr(
eventhdlr, "LPstat", "generate LP statistics after every LP event"
        )

#model.setParam("heuristics/shiftandpropagate/cutoffbreaker ","0") # number of cutoff before heurist stops

# Using set param different option of the solver can be setted
model.setParam("separating/maxrounds", "0")
model.setParam("separating/maxroundsroot", "0")
model.setParam("separating/maxcuts", "0")
model.setParam("separating/maxcutsroot", "0")
model.setParam("separating/maxstallrounds", "0")
model.setParam("separating/maxstallroundsroot", "0")
#model.setParam("conflict/enable","0")

#model.setParam("separating/clique/freq","-1")
#model.setParam("separating/flowcover/freq","-1")
#model.setParam("separating/cmir/freq","-1")
#model.setParam("separating/knapsackcover/freq","-1")
#model.setParam("separating/aggregation/freq","-1")
#model.setParam("separating/disjunctive/freq","-1")
#model.setParam("separating/gomory/freq","-1")
#model.setParam("separating/strongcg/freq","-1")
#model.setParam("separating/gomorymi/freq","-1")
#model.setParam("separating/impliedbounds/freq","-1")
#model.setParam("separating/mcf/freq ","-1")
#model.setParam("separating/minor/freq","-1")
#model.setParam("separating/mixing/freq ","-1")
# model.setParam("separating/rapidlearning/freq","-1")
#model.setParam("separating/rlt/freq ","-1")
#model.setParam("separating/zerohalf/freq","-1")
#model.setParam("presolving/maxrounds","0")
gallo=model.getParams()
#model.writeProblem('/home/luirusso/OA3.mps')

try:
    model.optimize()
except:
    print("optimization failed")
nodenumber=[]
parents=[]




##columns = nodelist[0].keys()
#df = pd.DataFrame(nodelist, columns=columns)



In [4]:
N=30
N_obs_max=5
binary_sol={}
b_l_sol, b_u_sol=[],[]
for i in range(2):
    b_l_sol.append([])
    b_u_sol.append([])
    for j in range(N):
        b_l_sol[i].append([])
        b_u_sol[i].append([])
        for k in range(N_obs_max):
            b_l_sol[i][j].append(m.getVal(bl[i,j,k]))    
            b_u_sol[i][j].append(m.getVal(bu[i,j,k]))
            binary_sol[bl[i,j,k].name]=m.getVal(bl[i,j,k])
            binary_sol[bu[i,j,k].name]=m.getVal(bu[i,j,k])

keys_bl=list(bl.keys())
bl_sol={}

keys_bu=list(bu.keys())

    


In [5]:
for i in range(len(nodelist)):
    parents.append(nodelist[i]["parent"])
    nodenumber.append(nodelist[i]["number"])
#nodenumber=nodenumber[2:len(nodenumber)-1]
number_set=set(nodenumber)
parents_set=set(parents)
#leaves_set=number_set-parents_set
#leaves_list=list(leaves_set)
#leaves_list=list(nodelist[len(nodelist)-1]["openNodesNumber"])
#leaves_set=set(leaves_list)

In [6]:
OpenNodes_perIteration=[]
for i in range(len(nodelist)): OpenNodes_perIteration.append(nodelist[i]["openNodesNumber"])
allOpenNodes=np.array([0])
for i in (range(len(OpenNodes_perIteration))): allOpenNodes=np.append(allOpenNodes,np.array(OpenNodes_perIteration[i]))
allOpenNodes=allOpenNodes.astype(int)
allOpenNodes_set=set(allOpenNodes)
#leaves_set=allOpenNodes_set&number_set
leaves_set=(number_set-parents_set)#{1,19,23,24,37,58,61,62,66,71,75,78}
leaves_list=list(leaves_set)


In [7]:
leaves_bounds={}
for i in range(len(nodelist)): 
    if nodelist[i]["number"] in leaves_set:# and nodelist[i]["number"] not in set(leaves_bounds.keys()):
        leaves_bounds[nodelist[i]["number"]]=[nodelist[i]["lb"], nodelist[i]["ub"]]
#

In [ ]:


lb_tilde={}
ub_tilde={}
binary_dim=len(leaves_bounds[leaves_list[1]][1].keys())
for b in range(binary_dim):
    lb_tilde[b]=1
    ub_tilde[b]=0
    for l in leaves_set:
        keys_lb=list(leaves_bounds[l][0].keys())
        keys_ub=list(leaves_bounds[l][1].keys())
        lb_tilde[b]=(min(lb_tilde[b],leaves_bounds[l][0][keys_lb[b]][1]))
        ub_tilde[b]=(max(ub_tilde[b],leaves_bounds[l][1][keys_ub[b]][1]))




In [ ]:
# solution checker
leaves_solution = [n['LPsol'] for n in nodelist]# if n['number'] in leaves_set]
for l in leaves_solution:
    if l==binary_sol:
        print('ok')
        


In [ ]:
#Covarage check
a=0
b=0
k=0
for i in range(len(ub_tilde)):
    a+=ub_tilde[i]
    b+=lb_tilde[i]
    if ub_tilde[i]==lb_tilde[i]:
        k=k+1
        print(keys_lb[i])
        print(keys_ub[i])
        print(k)


a

In [ ]:
python -m pip install treed


In [ ]:
#for i in range(len(nodelist)):
print(nodelist)

In [ ]:
from treed import TreeD
treed = TreeD(
    probpath="/home/luirusso/LearnWSL/model.cip",
    nodelimit=2000,
    scip_settings=[("separating/maxcuts", 0),("separating/knapsackcover/freq",-1)],
    showcuts=True,
    setfile="/home/luirusso/Params.txt"
)

treed.solve()
fig = treed.draw2d()
fig.show(renderer='notebook')
